In [1]:
import underworld3 as uw
import petsc4py
from petsc4py import PETSc
import gmsh
import sympy
import numpy as np


In [2]:
S2mesh = uw.meshing.CubedSphere(radiusOuter=1.0, radiusInner=0.98, numElements=21, simplex=True)

In [4]:
L = S2mesh.dm.getLabel("Upper1")
if L:
    print("Valid")
else:
    print("Not Valid")

Not Valid


petsc4py.PETSc.DMBoundaryType

In [3]:

# if uw.mpi.size==1:

import numpy as np
import pyvista as pv
import vtk

pv.global_theme.background = 'white'
pv.global_theme.window_size = [750, 250]
pv.global_theme.antialiasing = True
pv.global_theme.jupyter_backend = 'panel'
pv.global_theme.smooth_shading = True

S2mesh.vtk("tmp_S2_5.vtk")
pvmesh = pv.read("tmp_S2_5.vtk")
clipped = pvmesh.clip('y')


#     arrow_loc = np.zeros((v_soln.coords.shape[0],3))
#     arrow_loc[:,0:2] = v_soln.coords[...]

#     arrow_length = np.zeros((v_soln.coords.shape[0],3))
#     arrow_length[:,0:2] = vsol[...] 

pl = pv.Plotter()

pl.add_mesh(clipped,'Black', 'wireframe')



# pl.add_mesh(pvmesh, cmap="coolwarm", edge_color="Black", show_edges=True, scalars="dT2",
#               use_transparency=False, opacity=1)

#     pl.add_arrows(arrow_loc, arrow_length, mag=1.0e-1, opacity=0.5)
#pl.add_arrows(arrow_loc2, arrow_length2, mag=1.0e-1)

# pl.add_points(pdata)

pl.show(cpos="xy")

In [5]:
# Make a uw mesh from this ... 

x,y,z = S2mesh.X

hyp_xy  = sympy.sqrt(x**2 + y**2) 
hyp_xyz = sympy.sqrt(x**2 + y**2 + z**2) 

th = sympy.atan2(y,x)
ph = sympy.atan2(z, hyp_xy)


In [8]:
hyp_xyz.diff(x)

N.x/sqrt(N.x**2 + N.y**2 + N.z**2)

In [7]:
phi  = uw.discretisation.MeshVariable(r"\phi",       S2mesh , 1, vtype=uw.VarType.SCALAR, degree=1)
pi1  = uw.discretisation.MeshVariable(r"\Pi^{(1)}",  S2mesh , 1, vtype=uw.VarType.SCALAR, degree=1)
pi2  = uw.discretisation.MeshVariable(r"\Pi^{(2)}",  S2mesh , 1, vtype=uw.VarType.SCALAR, degree=1)

In [8]:
th_array = uw.function.evaluate(th, phi.coords)
ph_array = uw.function.evaluate(ph, phi.coords)
th_ph_coords = np.vstack((th_array, ph_array)).T

In [9]:
e_l1 = sympy.Matrix([-y, x, 0]) / hyp_xy  # unit vector, longitude
e_l2 = sympy.Matrix([-x*z,-y*z,x**2+y**2]) / (hyp_xyz*hyp_xy)


In [10]:
grad_phi = phi.sym.jacobian(S2mesh.X)

In [11]:
e_l1.dot(grad_phi) * sympy.cos(ph)

sqrt(N.x**2 + N.y**2)*(N.x*\phi_{,1}(N.x, N.y, N.z)/sqrt(N.x**2 + N.y**2) - N.y*\phi_{,0}(N.x, N.y, N.z)/sqrt(N.x**2 + N.y**2))/sqrt(N.x**2 + N.y**2 + N.z**2)

In [12]:
e_l2.dot(grad_phi) 

-N.x*N.z*\phi_{,0}(N.x, N.y, N.z)/(sqrt(N.x**2 + N.y**2)*sqrt(N.x**2 + N.y**2 + N.z**2)) - N.y*N.z*\phi_{,1}(N.x, N.y, N.z)/(sqrt(N.x**2 + N.y**2)*sqrt(N.x**2 + N.y**2 + N.z**2)) + sqrt(N.x**2 + N.y**2)*\phi_{,2}(N.x, N.y, N.z)/sqrt(N.x**2 + N.y**2 + N.z**2)

In [13]:
scalar_projection = uw.systems.Projection(S2mesh, pi2)
scalar_projection.uw_function = e_l1.dot(grad_phi) * sympy.cos(ph)
scalar_projection.smoothing = 1.0e-3
scalar_projection.petsc_options["snes_rtol"] = 1.0e-6

In [14]:
with S2mesh.access(phi):
    phi.data[:,0] = np.sin(3*th_array[:])

In [15]:
scalar_projection.solve()
with S2mesh.access(pi1):
    pi1.data[...] = pi2.data[...]

  0 SNES Function norm 0.00720144 
    Residual norms for SProj_1_ solve.
    0 KSP Residual norm 7.201440209888e-03 
    1 KSP Residual norm 1.944531261247e-04 
    2 KSP Residual norm 1.573940461886e-05 
    3 KSP Residual norm 1.216401591777e-06 
  1 SNES Function norm 1.2164e-06 
    Residual norms for SProj_1_ solve.
    0 KSP Residual norm 1.216401591777e-06 
    1 KSP Residual norm 1.435942139199e-07 
    2 KSP Residual norm 1.214842067146e-08 
    3 KSP Residual norm 9.650965043369e-10 
  2 SNES Function norm 9.651e-10 
Nonlinear SProj_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2


In [16]:


scalar_projection.uw_function = e_l2.dot(grad_phi) * sympy.cos(ph)**4
scalar_projection.solve()

  0 SNES Function norm 3.24411e-07 
    Residual norms for SProj_1_ solve.
    0 KSP Residual norm 3.244114662775e-07 
    1 KSP Residual norm 2.808884478834e-08 
    2 KSP Residual norm 2.226339752308e-09 
    3 KSP Residual norm 1.789001342933e-10 
  1 SNES Function norm 1.789e-10 
    Residual norms for SProj_1_ solve.
    0 KSP Residual norm 1.789001342933e-10 
    1 KSP Residual norm 2.113991923024e-11 
    2 KSP Residual norm 1.707098952765e-12 
    3 KSP Residual norm 1.516077740069e-13 
  2 SNES Function norm < 1.e-11
Nonlinear SProj_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2


In [ ]:
# poisson = uw.systems.Poisson(S2mesh,
#                              u_Field=phi,
#                              degree=2,
#                              solver_name="diffusion")

# # Constitutive law (diffusivity)

# poisson.constitutive_model = uw.systems.constitutive_models.DiffusionModel(S2mesh.dim)
# poisson.constitutive_model.material_properties = poisson.constitutive_model.Parameters(diffusivity = 1)

# poisson.f = x**2

In [ ]:
# poisson.solve()

In [18]:

if uw.mpi.size==1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = 'white'
    pv.global_theme.window_size = [750, 250]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = 'panel'
    pv.global_theme.smooth_shading = True
        
    S2mesh.vtk("latlonmesh.vtk")
    pvmesh = pv.read("latlonmesh.vtk")

    # with meshbox.access():
    #     vsol = v_soln.data.copy()
  
    with S2mesh.access():
        pvmesh.point_data["T"]    = uw.function.evaluate(phi.sym[0], S2mesh.data)
        pvmesh.point_data["dT1"]  = pi1.data[:,0]
        pvmesh.point_data["dT2"]  = pi2.data[:,0]


#     arrow_loc = np.zeros((v_soln.coords.shape[0],3))
#     arrow_loc[:,0:2] = v_soln.coords[...]
    
#     arrow_length = np.zeros((v_soln.coords.shape[0],3))
#     arrow_length[:,0:2] = vsol[...] 

    clipped = pvmesh.clip('y')


   
    pl = pv.Plotter()
    
    pl.add_mesh(clipped, cmap="coolwarm", edge_color="Black", show_edges=True, scalars="dT1",
                  use_transparency=False, opacity=1)
    
#     pl.add_arrows(arrow_loc, arrow_length, mag=1.0e-1, opacity=0.5)
    #pl.add_arrows(arrow_loc2, arrow_length2, mag=1.0e-1)
    
    # pl.add_points(pdata)

    pl.show(cpos="xy")

In [19]:
poisson = uw.systems.Poisson(S2mesh,
                             u_Field=phi,
                             solver_name="diffusion")

# Constitutive law (diffusivity)

poisson.constitutive_model = uw.systems.constitutive_models.DiffusionModel(S2mesh.dim)
poisson.constitutive_model.material_properties = poisson.constitutive_model.Parameters(diffusivity = 1)

poisson.f = 0.0
# poisson.petsc_options.delValue('ksp_monitor')


In [ ]:
poisson.solve()

  0 SNES Function norm 0.00155581 
    Residual norms for diffusion_ solve.
    0 KSP Residual norm 1.555810117734e-03 
    1 KSP Residual norm 1.405049461342e-03 
    2 KSP Residual norm 1.402423771278e-03 
    3 KSP Residual norm 1.397741538487e-03 
    4 KSP Residual norm 5.095392281200e-04 
    5 KSP Residual norm 7.078175704890e-06 
    6 KSP Residual norm 5.121725003706e-08 
  1 SNES Function norm 0.00155581 
    Residual norms for diffusion_ solve.
    0 KSP Residual norm 1.555809658215e-03 
    1 KSP Residual norm 1.404538370652e-03 
    2 KSP Residual norm 1.402392410586e-03 
    3 KSP Residual norm 1.399209998802e-03 
    4 KSP Residual norm 7.597383207105e-04 


In [ ]:
## Suppose we try a vector problem (Stokes) with rotated degrees of freedom

In [18]:
import gmsh
gmsh.initialize()
gmsh.model.add("Manifold")

sphere_tag = gmsh.model.occ.add_sphere(0, 0, 0, 1.0)
gmsh.model.occ.synchronize()
surfaces = gmsh.model.getEntities(2)



In [19]:
gmsh.model.mesh.generate(2)
gmsh.write("tmp_2sphere.msh")
gmsh.finalize()
plex = PETSc.DMPlex().createFromFile("tmp_2sphere.msh")

Info    : Meshing 1D...
Info    : [ 40%] Meshing curve 2 (Circle)
Info    : Done meshing 1D (Wall 0.000688792s, CPU 0.000585s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0206938s, CPU 0.016742s)
Info    : 164 nodes 336 elements
Info    : Writing 'tmp_2sphere.msh'...
Info    : Done writing 'tmp_2sphere.msh'


In [20]:
plex.view()

DM Object: unnamed 1 MPI process
  type: plex
unnamed in 2 dimensions:
  Number of 0-cells per rank: 164
  Number of 1-cells per rank: 486
  Number of 2-cells per rank: 324
Labels:
  celltype: 3 strata with value/size (0 (164), 3 (324), 1 (486))
  depth: 3 strata with value/size (0 (164), 1 (486), 2 (324))


[(2, 1)]